In [22]:
from crewai import Agent
from crewai_tools import MCPServerAdapter
from mcp import StdioServerParameters

In [23]:
import os
import json
import pytz
from crewai import Agent, Task, Crew, Process, LLM

In [24]:
server_params=StdioServerParameters(
    command="python3",
    args=["server.py"],
    env={"UV_PYTHON": "3.12", **os.environ},
)


In [25]:
llm = LLM(
    model="bedrock/us.anthropic.claude-3-7-sonnet-20250219-v1:0",
    aws_region_name="us-east-1",
    temperature=0.2,        # Higher for more creative outputs
)

In [26]:
with MCPServerAdapter(server_params) as mcp_tools:
    print(f"Available tools: {[tool.name for tool in mcp_tools]}")


Exception in thread Thread-48 (_run_loop):
  + Exception Group Traceback (most recent call last):
  |   File "/usr/lib/python3.12/threading.py", line 1073, in _bootstrap_inner
  |     self.run()
  |   File "/home/test/excel_reader/.venv/lib/python3.12/site-packages/ipykernel/ipkernel.py", line 772, in run_closure
  |     _threading_Thread_run(self)
  |   File "/usr/lib/python3.12/threading.py", line 1010, in run
  |     self._target(*self._args, **self._kwargs)
  |   File "/home/test/excel_reader/.venv/lib/python3.12/site-packages/mcpadapt/core.py", line 237, in _run_loop
  |     self.loop.run_until_complete(self.task)
  |   File "/usr/lib/python3.12/asyncio/base_events.py", line 687, in run_until_complete
  |     return future.result()
  |            ^^^^^^^^^^^^^^^
  |   File "/home/test/excel_reader/.venv/lib/python3.12/site-packages/mcpadapt/core.py", line 226, in setup
  |     await stack.enter_async_context(
  |   File "/usr/lib/python3.12/contextlib.py", line 659, in enter_async

RuntimeError: Failed to initialize MCP Adapter: Couldn't connect to the MCP server after 30 seconds

In [ ]:
excel_analyze = Agent(
        role='Excel Data Processor',
        goal='Extract clean tabular data from Excel files by filtering out garbage, headers, and formatting',
        backstory="""You are an expert data processor specializing in cleaning Excel files. 
        You can identify and extract only the meaningful tabular data from messy Excel files 
        that contain extra headings, formatting, and other unwanted elements. Your job is to 
        produce clean, structured data that can be easily used for analysis.""",
        tools= mcp_tools,
        verbose=True,
        allow_delegation=False,
        llm = llm
    )

SyntaxError: incomplete input (503878995.py, line 11)

In [ ]:
excel_analyzer = Task(
        description=f"""
        Process the Excel file located at '{input_file}' and extract only the clean tabular data.
        
        Your task is to:
        1. Connect to the MCP server to read the Excel file
        2. Identify and extract only the meaningful tabular data
        3. Filter out any garbage data such as:
           - Extra headers and titles
           - Formatting rows
           - Empty rows and columns
           - Merged cells content that's not part of the data table
           - Summary rows or totals that are not part of the main data
        4. Save the cleaned data to '{output_file}'
        
        Use the mcp_excel_processor tool to accomplish this task.
        """,
        expected_output=f"A clean Excel file saved at '{output_file}' containing only the structured tabular data without any formatting or garbage content.",
        agent= excel_analyze
    )

In [ ]:
crew = Crew(
    agents=[excel_analyze],
    tasks=[excel_analyzer],
    verbose=2
)